In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("merged_data.csv")

In [3]:
df.head()

,name,id,type,classification,mass,fall,year,latitude,longitude,country,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Haven,11858,Valid,H6,6100.0,Found,1950.0,37.96417,-97.75583,United States,...,25.239417,25.479003,25.713686,25.939171,26.155144,26.323940,26.445826,26.590781,26.769253,26.948271
1,St. Louis,23089,Valid,H4,1000.0,Fell,1950.0,38.70000,-90.23333,United States,...,25.239417,25.479003,25.713686,25.939171,26.155144,26.323940,26.445826,26.590781,26.769253,26.948271
2,Arroyo Aguiar,2340,Valid,H5,7450.0,Fell,1950.0,-31.41667,-60.66667,Argentina,...,15.573055,15.724678,15.863864,15.994957,16.109071,16.187335,16.230432,16.264683,16.311426,16.367933
3,Plainview (1950),18842,Valid,H,2200.0,Found,1950.0,34.11667,-101.78333,United States,...,25.239417,25.479003,25.713686,25.939171,26.155144,26.323940,26.445826,26.590781,26.769253,26.948271
4,Santa Rosalia,23168,Valid,"Pallasite, PMG",1631.0,Found,1950.0,27.33333,-112.33333,Mexico,...,61.795294,62.397078,62.983378,63.582411,64.189415,64.718226,65.151603,65.644123,66.219161,66.791446


In [4]:
# Convert year to integer
df['year'] = df['year'].astype(int)

# Extract population density for the year of meteorite landing
df['population_density'] = df.apply(
    lambda row: row[str(row['year'])] if str(row['year']) in df.columns else None, axis=1
)

In [5]:
# Encode country as a categorical variable
label_encoder = LabelEncoder()
df['country_encoded'] = label_encoder.fit_transform(df['country'])

In [6]:
df['fall_encoded'] = label_encoder.fit_transform(df['fall'])

In [7]:
# Drop columns from 1950 to 2024
columns_to_drop = [str(year) for year in range(1950, 2025)]
df.drop(columns=columns_to_drop, inplace=True)

In [8]:
df.head()

,name,id,type,classification,mass,fall,year,latitude,longitude,country,population_density,country_encoded,fall_encoded
0,Haven,11858,Valid,H6,6100.0,Found,1950,37.96417,-97.75583,United States,8.628275,91,1
1,St. Louis,23089,Valid,H4,1000.0,Fell,1950,38.70000,-90.23333,United States,8.628275,91,0
2,Arroyo Aguiar,2340,Valid,H5,7450.0,Fell,1950,-31.41667,-60.66667,Argentina,6.095700,3,0
3,Plainview (1950),18842,Valid,H,2200.0,Found,1950,34.11667,-101.78333,United States,8.628275,91,1
4,Santa Rosalia,23168,Valid,"Pallasite, PMG",1631.0,Found,1950,27.33333,-112.33333,Mexico,14.080882,51,1


In [9]:
# Create target: count of meteorites found in each country-year combination
meteorite_count = df.groupby(['country', 'year']).size().reset_index(name='count')

In [10]:
# Merge count back with original dataset
df = pd.merge(df, meteorite_count, on=['country', 'year'], how='left')

In [11]:
# Handle missing population density data (can be filled or dropped)
df = df.dropna(subset=['population_density'])

In [12]:
df.head()

,name,id,type,classification,mass,fall,year,latitude,longitude,country,population_density,country_encoded,fall_encoded,count
0,Haven,11858,Valid,H6,6100.0,Found,1950,37.96417,-97.75583,United States,8.628275,91,1,19
1,St. Louis,23089,Valid,H4,1000.0,Fell,1950,38.70000,-90.23333,United States,8.628275,91,0,19
2,Arroyo Aguiar,2340,Valid,H5,7450.0,Fell,1950,-31.41667,-60.66667,Argentina,6.095700,3,0,1
3,Plainview (1950),18842,Valid,H,2200.0,Found,1950,34.11667,-101.78333,United States,8.628275,91,1,19
4,Santa Rosalia,23168,Valid,"Pallasite, PMG",1631.0,Found,1950,27.33333,-112.33333,Mexico,14.080882,51,1,2


In [13]:
# Define features and target
X = df[['year', 'mass', 'fall_encoded', 'latitude', 'longitude', 'country_encoded', 'population_density', 'count']]

In [14]:
# Function to split the data based on country and year with at least 'min_test_size' events in total
def custom_train_test_split(df, n_splits=1, min_test_size=800):
    splits = []
    
    for _ in range(n_splits):
        # Ensure the test set has at least 'min_test_size' events in total
        test_data = pd.DataFrame()  # Initialize empty DataFrame for test data
        
        while len(test_data) < min_test_size:
            # Pick a random country and year combination for the test set
            test_country = np.random.choice(df['country_encoded'].unique())
            test_year = np.random.choice(df[df['country_encoded'] == test_country]['year'].unique())
            
            # Filter test data for that specific country and year
            new_test_data = df[(df['country_encoded'] == test_country) & (df['year'] == test_year)]
            
            # Add the new data to the test set, avoiding duplicates
            test_data = pd.concat([test_data, new_test_data]).drop_duplicates()
        
        # Filter training data to exclude the test country and year
        train_data = df[~df.index.isin(test_data.index)]
        
        splits.append((train_data, test_data))
    
    return splits

In [15]:
# Function to recover features and target from the splits
def recover_features_and_target(train_data, test_data, target_column='count'):
    # Separate features and target for both train and test
    X_train = train_data.drop(columns=[target_column])
    y_train = train_data[target_column]
    X_test = test_data.drop(columns=[target_column])
    y_test = test_data[target_column]
    
    return X_train, X_test, y_train, y_test

In [30]:
splits = custom_train_test_split(X, n_splits=1)

In [31]:
# Example usage for each split:
for i, (train, test) in enumerate(splits):
    print(f"Processing Split {i + 1}")
    X_train, X_test, y_train, y_test = recover_features_and_target(train, test, target_column='count')
    
    # Example: printing the shapes of the datasets
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"y_test shape: {y_test.shape}")
    print("\n")

Processing Split 1
X_train shape: (6992, 7)
X_test shape: (1180, 7)
y_train shape: (6992,)
y_test shape: (1180,)




In [32]:
# Check how many unique values are present in each column
unique_values_per_column = X_test.nunique()

# Print the result
print(unique_values_per_column)

year                    60
mass                  1011
fall_encoded             2
latitude               927
longitude              928
country_encoded         93
population_density     183
dtype: int64


# Linear Regression

In [33]:
# Linear Regression Model
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [34]:
# Predict on test set
y_pred_lr = lr.predict(X_test)

In [35]:
# Evaluate model
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"Linear Regression Mean Squared Error: {mse_lr}")

Linear Regression Mean Squared Error: 18387.39283312726


# Random Forest

In [36]:
# Random Forest Regressor Model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [37]:
# Predict on test set
y_pred_rf = rf.predict(X_test)

In [38]:
# Evaluate model
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest Mean Squared Error: {mse_rf}")

Random Forest Mean Squared Error: 6367.599964406779


# Mixed effect Model

In [39]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [90]:
# Fit the Mixed Effects Model with count as target
mixed_model = smf.mixedlm("count ~ year + population_density", X, groups=X["country_encoded"])
mixed_model_fitted = mixed_model.fit()

In [91]:
# Print model summary
print(mixed_model_fitted.summary())

               Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    count      
No. Observations:     8172       Method:                REML       
No. Groups:           96         Scale:                 14210.3680 
Min. group size:      1          Log-Likelihood:        -50721.4699
Max. group size:      2992       Converged:             Yes        
Mean group size:      85.1                                         
-------------------------------------------------------------------
                    Coef.   Std.Err.   z    P>|z|  [0.025   0.975] 
-------------------------------------------------------------------
Intercept          2004.374  286.499  6.996 0.000 1442.846 2565.903
year                 -0.998    0.145 -6.871 0.000   -1.282   -0.713
population_density   -0.043    0.100 -0.427 0.669   -0.239    0.154
Group Var          3895.307    6.035                               



In [92]:
# Now, let's predict the number of meteorites for the test set
# We need to make predictions on the test set by using the fitted model
X_test['predicted_count'] = mixed_model_fitted.predict(X_test)

In [93]:
# Evaluate the predicted counts
from sklearn.metrics import mean_squared_error
mse_mixed = mean_squared_error(y_test, X_test['predicted_count'])
print(f"Mixed Effect Model Mean Squared Error: {mse_mixed}")

Mixed Effect Model Mean Squared Error: 56577.23958635178


# Just Fell

In [47]:
# Filter the DataFrame to only include rows where "fall" == "fell"
filtered_df = df[df['fall'] == 'Fell']

In [48]:
filtered_df

,name,id,type,classification,mass,fall,year,latitude,longitude,country,population_density,country_encoded,fall_encoded,count
1,St. Louis,23089,Valid,H4,1000.0,Fell,1950,38.70000,-90.23333,United States,8.628275,91,0,19
2,Arroyo Aguiar,2340,Valid,H5,7450.0,Fell,1950,-31.41667,-60.66667,Argentina,6.095700,3,0,1
5,Vengerovo,24158,Valid,H5,10800.0,Fell,1950,56.13333,77.26667,Russia,6.313780,69,0,1
6,Monte das Fortes,16725,Valid,L5,4885.0,Fell,1950,38.01667,-8.25000,Portugal,91.524450,66,0,1
15,Madhipura,15379,Valid,L,1000.0,Fell,1950,25.91667,86.36667,India,116.467101,34,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35524,Sołtmany,53829,Valid,L6,1066.0,Fell,2011,54.00883,22.00500,Poland,124.785732,65,0,1
35595,Boumdeid (2011),57167,Valid,L6,3599.0,Fell,2011,17.17493,-11.34133,Mauritania,3.398478,50,0,1
36217,Battle Mountain,56133,Valid,L6,2900.0,Fell,2012,40.66813,-117.18913,United States,24.506701,91,0,8
36257,Sutter's Mill,55529,Valid,C,992.5,Fell,2012,38.80389,-120.90806,United States,24.506701,91,0,8


In [67]:
# Define features and target
X_fell = filtered_df[['year', 'mass', 'latitude', 'longitude', 'country_encoded', 'population_density', 'count']]

In [80]:
splits_fell = custom_train_test_split(X_fell, n_splits=1, min_test_size = 25 )

In [81]:
# Example usage for each split:
for i, (train_fell, test_fell) in enumerate(splits_fell):
    print(f"Processing Split {i + 1}")
    X_train_fell, X_test_fell, y_train_fell, y_test_fell = recover_features_and_target(train, test, target_column='count')
    
    # Example: printing the shapes of the datasets
    print(f"X_train shape: {X_train_fell.shape}")
    print(f"X_test shape: {X_test_fell.shape}")
    print(f"y_train shape: {y_train_fell.shape}")
    print(f"y_test shape: {y_test_fell.shape}")
    print("\n")

Processing Split 1
X_train shape: (325, 6)
X_test shape: (25, 6)
y_train shape: (325,)
y_test shape: (25,)




In [82]:
# Check how many unique values are present in each column
unique_values_per_column = X_test_fell.nunique()

# Print the result
print(unique_values_per_column)

year                  17
mass                  24
latitude              25
longitude             25
country_encoded       22
population_density    24
dtype: int64


## Linear Regression

In [83]:
# Linear Regression Model
lr_fell = LinearRegression()
lr_fell.fit(X_train_fell, y_train_fell)

LinearRegression()

In [84]:
# Predict on test set
y_pred_lr_fell = lr_fell.predict(X_test_fell)

In [85]:
# Evaluate model
mse_lr_fell = mean_squared_error(y_test_fell, y_pred_lr_fell)
print(f"Linear Regression Mean Squared Error: {mse_lr_fell}")

Linear Regression Mean Squared Error: 18.26838891868307


## Random Forest

In [86]:
# Random Forest Regressor Model
rf_fell = RandomForestRegressor(n_estimators=100, random_state=42)
rf_fell.fit(X_train_fell, y_train_fell)

RandomForestRegressor(random_state=42)

In [87]:
# Predict on test set
y_pred_rf_fell = rf_fell.predict(X_test_fell)

In [88]:
# Evaluate model
mse_rf_fell = mean_squared_error(y_test_fell, y_pred_rf_fell)
print(f"Random Forest Mean Squared Error: {mse_rf_fell}")

Random Forest Mean Squared Error: 19.73948


## Mixed effect

In [94]:
# Fit the Mixed Effects Model with count as target
mixed_model_fell = smf.mixedlm("count ~ year + population_density", X_fell, groups=X_fell["country_encoded"])
mixed_model_fitted_fell = mixed_model_fell.fit()

In [95]:
# Print model summary
print(mixed_model_fitted_fell.summary())

             Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   count     
No. Observations:     350       Method:               REML      
No. Groups:           81        Scale:                21.2081   
Min. group size:      1         Log-Likelihood:       -1135.3721
Max. group size:      45        Converged:            Yes       
Mean group size:      4.3                                       
----------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
----------------------------------------------------------------
Intercept          -44.274   36.297 -1.220 0.223 -115.416 26.867
year                 0.024    0.018  1.295 0.195   -0.012  0.060
population_density  -0.006    0.008 -0.717 0.473   -0.022  0.010
Group Var           93.441    4.604                             



In [96]:
# Now, let's predict the number of meteorites for the test set
# We need to make predictions on the test set by using the fitted model
X_test_fell['predicted_count'] = mixed_model_fitted_fell.predict(X_test_fell)

In [97]:
# Evaluate the predicted counts
from sklearn.metrics import mean_squared_error
mse_mixed_fell = mean_squared_error(y_test_fell, X_test_fell['predicted_count'])
print(f"Mixed Effect Model Mean Squared Error: {mse_mixed_fell}")

Mixed Effect Model Mean Squared Error: 2.774419218640848
